# Loan Prediction – Model Evaluation and Selection

This notebook evaluates the performance of trained machine learning models
using appropriate classification metrics. The best-performing model is
selected based on quantitative results.

## 1. Import Libraries and Load Cleaned Dataset

The cleaned dataset is loaded to ensure consistency with the preprocessing
and modeling stages.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    roc_curve
)